In [1]:
# Loading all needed packages at the start
import elastix 
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import os
import SimpleITK as sitk
import numpy as np
import shutil

from FunctionsImageRegistration import *


## Fill in your own paths in the cell below ##

In [2]:
# to Elastix.exe
ELASTIX_PATH = 'C:/Users/20183303/Documents/Jaar 5 - ME/Q3/CS in medische beeldanalyse/Elastix files/Elastix.exe'
el = elastix.ElastixInterface(elastix_path=ELASTIX_PATH)
# to Transformix.exe
TRANSFORMIX_PATH = 'C:/Users/20183303/Documents/Jaar 5 - ME/Q3/CS in medische beeldanalyse/Elastix files/Transformix.exe'
# to TrainingData
filepath_data = r'C:\Users\20183303\Documents\Jaar 5 - ME\Q3\CS in medische beeldanalyse\Image Registration Assignment\TrainingDataIR'


### Default settings


In [3]:
# default settings for MI computation
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['image.interpolation'] = 'nearest'


#### TO DO: Test out hypothesis: does the nr of bins in the MI computation function ever influence the order of the highest MI values?

## Per manually selected fixed image, find the five most similar moving image slices and perform B-spline transformation on them:


In [4]:
# Test set containing p119, p128, and p135 are left out!

patient_nrs_list = ['p102','p107', 'p108', 'p109', 'p115', 'p116', 'p117', 'p120', 'p125', 'p127', 'p129', 'p133']
#                    0      1       2       3       4       5       6       7       8       9       10      11       

# load all 12 patients' data (every 3D array) in one list, which has the same order as 'patient_nrs_list'
loaded_mr_list = []
for value in patient_nrs_list:
    loaded_mr, __ = loadPatientDataReadable(value, filepath=filepath_data)
    loaded_mr_list.append(loaded_mr)
      
# select (for now manually) which image is currently your fixed image -- # Enter image level of one fixed image
fixed_image_index = 5 # index in patient_nrs_list
patient_id_fixed_image = patient_nrs_list[fixed_image_index] # patient ID (for example 'p102') of fixed image
fixed_image_path = filepath_data+ '\{}\mr_bffe.mhd'.format(patient_id_fixed_image) # path to fixed image

print('Patient selected as fixed image:', patient_id_fixed_image)

# select all but fixed image to use as moving image
train_mr_list_moving = []
for j in loaded_mr_list:
    train_mr_list_moving.append(j)
    
fixed_image_mr = train_mr_list_moving.pop(fixed_image_index) # 3D array of fixed image

# # # # Enter slice iterator of fixed image # # # # 

for image_slice_fixed in range(4): # when everything works: use 86. When everything is still being tested: keep it simple with 4. 
    
    print('Fixed image slice nr:', image_slice_fixed)

    list_of_list_MI, list_moving_slices_used = allMImovingImages(train_mr_list_moving, image_slice_fixed, fixed_image_mr)

    per_patient_per_slice_MI = intoOneList(list_of_list_MI)

    most_similar_list_index = sort_index(per_patient_per_slice_MI)[:5] # Get the five most similar images of all moving images
    #print(most_similar_list_index)

    patient_ids_similar = []
    slice_ids_similar = []
    for j in most_similar_list_index:
        patient_index_a, actual_slice_nr_a = getPatientAndSlice(j, list_moving_slices_used, train_mr_list_moving, patient_nrs_list)
        patient_ids_similar.append(patient_nrs_list[patient_index_a])
        slice_ids_similar.append(actual_slice_nr_a)

    # # # # Generate 2D arrays for all patients that will be transformed # # # #
    
    # maybe add if-statement in here so arrays are only created if they do not exist already
    
    # # # # Enter slice iterator of moving images # # # #
    for moving_image_top_five in range(len(patient_ids_similar)):
        patient_id_moving_image = patient_ids_similar[moving_image_top_five]
        moving_image_path = filepath_data+ '\{}\mr_bffe.mhd'.format(patient_id_moving_image)
        create2DImages(fixed_image_path, moving_image_path, patient_id_fixed_image, patient_id_moving_image, filepath_data)

    print('Done creating 2D arrays')
    
    # Perform B-spline registration for each most similar slice and store the results
    # # # # Enter iterator for most similar slices (paired with correct patient) # # # #
    for work_down_list in range(len(patient_ids_similar)): # 0 to 5
        bspline_registration(patient_id_fixed_image, patient_ids_similar[work_down_list], image_slice_fixed, slice_ids_similar[work_down_list], filepath_data, el_path=el)
        
    ### You probably want to continue on this level: the slice level of the fixed images. 
    ### Now you have the five transformations that need to be combined with STAPLE.
    
# Continue with visualisation of B-spline transformation, STAPLE, Jacobian, transformation onto masks, and finally evaluation metrics.

Patient selected as fixed image: p108
Fixed image slice nr: 0
Patient: p127 and slice nr: 5
Patient: p127 and slice nr: 2
Patient: p127 and slice nr: 1
Patient: p127 and slice nr: 4
Patient: p127 and slice nr: 3
Done creating 2D arrays
C:/Users/20183303/Documents/Jaar 5 - ME/Q3/CS in medische beeldanalyse/Elastix files/Elastix.exe -f C:\Users\20183303\Documents\Jaar 5 - ME\Q3\CS in medische beeldanalyse\Image Registration Assignment\TrainingDataIR\p108\p108_slice0.mhd -m C:\Users\20183303\Documents\Jaar 5 - ME\Q3\CS in medische beeldanalyse\Image Registration Assignment\TrainingDataIR\p127\p127_slice5.mhd -p C:\Users\20183303\Documents\Jaar 5 - ME\Q3\CS in medische beeldanalyse\Image Registration Assignment\TrainingDataIR\parameters_penalty.txt -out C:\Users\20183303\Documents\Jaar 5 - ME\Q3\CS in medische beeldanalyse\Image Registration Assignment\TrainingDataIR\p108\bspline_results_slice_0\moving_slice_p127_5
Done performing B-splines for moving patient p127, slice 5
C:/Users/2018330

Done performing B-splines for moving patient p127, slice 5
C:/Users/20183303/Documents/Jaar 5 - ME/Q3/CS in medische beeldanalyse/Elastix files/Elastix.exe -f C:\Users\20183303\Documents\Jaar 5 - ME\Q3\CS in medische beeldanalyse\Image Registration Assignment\TrainingDataIR\p108\p108_slice2.mhd -m C:\Users\20183303\Documents\Jaar 5 - ME\Q3\CS in medische beeldanalyse\Image Registration Assignment\TrainingDataIR\p127\p127_slice4.mhd -p C:\Users\20183303\Documents\Jaar 5 - ME\Q3\CS in medische beeldanalyse\Image Registration Assignment\TrainingDataIR\parameters_penalty.txt -out C:\Users\20183303\Documents\Jaar 5 - ME\Q3\CS in medische beeldanalyse\Image Registration Assignment\TrainingDataIR\p108\bspline_results_slice_2\moving_slice_p127_4
Done performing B-splines for moving patient p127, slice 4
C:/Users/20183303/Documents/Jaar 5 - ME/Q3/CS in medische beeldanalyse/Elastix files/Elastix.exe -f C:\Users\20183303\Documents\Jaar 5 - ME\Q3\CS in medische beeldanalyse\Image Registration Ass

Done performing B-splines for moving patient p127, slice 1
C:/Users/20183303/Documents/Jaar 5 - ME/Q3/CS in medische beeldanalyse/Elastix files/Elastix.exe -f C:\Users\20183303\Documents\Jaar 5 - ME\Q3\CS in medische beeldanalyse\Image Registration Assignment\TrainingDataIR\p108\p108_slice4.mhd -m C:\Users\20183303\Documents\Jaar 5 - ME\Q3\CS in medische beeldanalyse\Image Registration Assignment\TrainingDataIR\p127\p127_slice3.mhd -p C:\Users\20183303\Documents\Jaar 5 - ME\Q3\CS in medische beeldanalyse\Image Registration Assignment\TrainingDataIR\parameters_penalty.txt -out C:\Users\20183303\Documents\Jaar 5 - ME\Q3\CS in medische beeldanalyse\Image Registration Assignment\TrainingDataIR\p108\bspline_results_slice_4\moving_slice_p127_3
Done performing B-splines for moving patient p127, slice 3
C:/Users/20183303/Documents/Jaar 5 - ME/Q3/CS in medische beeldanalyse/Elastix files/Elastix.exe -f C:\Users\20183303\Documents\Jaar 5 - ME\Q3\CS in medische beeldanalyse\Image Registration Ass